In [3]:
import pandas as pd
import jsonlines

df = pd.read_csv('examples_to_annotate.csv')
df['sample_id'] = df.index
skip_idx = []
data = []
# for pilot_file in ['pilot_bge_small.jsonl', 'pilot_openai_small.jsonl']:
#     with jsonlines.open(pilot_file) as reader:
#         for obj in reader:
#             data.append(obj)

# pilot_df = pd.DataFrame(data).drop_duplicates()
# skip_idx = pilot_df['sample_id'].values.tolist()


In [4]:
sports_idx = [0, 22, 23, 25, 27, 28, 33, 35, 37, 38, 39, 43, 44, 45, 48, 50, 51, 57, 58, 65, 68, 97, 98, 99, 100, 101, 102, 103, 107, 108, 115, 137, 138, 140, 142, 143, 148, 150, 152, 153, 154, 158, 159, 160, 163, 165, 166, 172, 173, 180, 183, 212, 213, 214, 215, 216, 217, 218, 222, 223, 230, 252, 253, 255, 257, 258, 263, 265, 267, 268, 269, 273, 274, 275, 278, 280, 281, 287, 288, 295, 298, 327, 328, 329, 330, 331, 332, 333, 337, 338, 345, 367, 368, 370, 372, 373, 378, 380, 382, 383, 384, 388, 389, 390, 393, 395, 396, 402, 403, 410, 413, 442, 443, 444, 445, 446, 447, 448, 452, 453, 460, 482, 483, 485, 487, 488, 493, 495, 497, 498, 499, 503, 504, 505, 508, 510, 511, 517, 518, 525, 528, 557, 558, 559, 560, 561, 562, 563, 567, 568, 575, 597, 598, 600, 602, 603, 608, 610, 612, 613, 614, 618, 619, 620, 623, 625, 626, 632, 633, 640, 643, 672, 673, 674, 675, 676, 677, 678, 682, 683, 690, 712, 713, 715, 717, 718, 723, 725, 727, 728, 729, 733, 734, 735, 738, 740, 741, 747, 748, 755, 758, 787, 788, 789, 790, 791, 792, 793, 797, 798, 805, 827, 828, 830, 832, 833, 838, 840, 842, 843, 844, 848, 849, 850, 853, 855, 856, 862, 863, 870, 873, 902, 903, 904, 905, 906, 907, 908, 912, 913, 922, 924, 925, 928, 936, 941, 945, 950, 955, 956, 957, 958, 959, 960, 961, 964, 971, 975, 977, 978, 1003, 1004, 1005, 1015, 1016, 1017, 1018, 1020, 1023, 1027, 1035, 1057, 1058, 1060, 1062, 1063, 1068, 1070, 1072, 1073, 1074, 1078, 1079, 1080, 1083, 1085, 1086, 1092, 1093, 1100, 1103, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1142, 1143]
skip_idx += sports_idx
skip_idx = set(skip_idx)

In [9]:
len(skip_idx)
17*50

850

In [6]:
def count_words(text):
    return len(str(text).split())

df_after_skip = df.drop(index=skip_idx)
df_after_skip['word_count'] = df_after_skip['source'].apply(count_words)

# Step 1: Group by source_doc and get one representative record for each source_doc (to get its word count)
sorted_df = df_after_skip.groupby('source').apply(lambda x: x.head(1))

# Step 2: Sort the source documents based on their word count
sorted_source_docs = sorted_df.sort_values(by='word_count')['source']
# print(sorted_source_docs.shape) # 115

# Step 3: Create batches of 10 records for each source document, sorted by word count of the source document
num_src_per_group = 5
dump_dir = "batch_5_src_no_sports/virgin_data"
for src_idx, doc in enumerate(sorted_source_docs):
    samples = df[df['source'] == doc]
    # print("batch", batch_idx)
    # print("index:", batch['sample_id'].values.tolist())
    batch_idx = src_idx // num_src_per_group + 1 # 1-indexed
    dump_file = f"{dump_dir}/batch_{batch_idx}.jsonl"
    with open(dump_file, mode="a") as f: 
        f.write(samples.to_json(orient='records', lines=True))

    # batch.to_json(f"batch_files//batch{batch_idx}.jsonl", orient='records', lines=True)

In [23]:
# Assign batches to annotators based on remainder division 

people = ["amin", "forrest", "rogger", "renyi", "miaoran", "weisi", "yujia", "suleman", "vivek", "x"]
# group people based on the index of them in the list `people`
people_grouped = [
    [person for j, person in enumerate(people) if j % 10 == i]
    for i in range(10)
]
people_grouped = [", ".join(group) for group in people_grouped]

print (people_grouped)
pd.DataFrame({"who  ": people_grouped, "server": [f"http://35.222.72.66:{9000+i}" for i in range(10)]})

['amin, x', 'forrest', 'rogger', 'renyi', 'miaoran', 'weisi', 'chenyu', 'yujia', 'suleman', 'vivek']


,who,server
0,"amin, x",http://35.222.72.66:9000
1,forrest,http://35.222.72.66:9001
2,rogger,http://35.222.72.66:9002
3,renyi,http://35.222.72.66:9003
4,miaoran,http://35.222.72.66:9004
5,weisi,http://35.222.72.66:9005
6,chenyu,http://35.222.72.66:9006
7,yujia,http://35.222.72.66:9007
8,suleman,http://35.222.72.66:9008
9,vivek,http://35.222.72.66:9009


In [ ]:
# ingest individual batches 
